**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

Answer: 

Function act: at each episode of a game, the environment class takes the output of act function: based on the action taken, the environment changes and the games evolves. The act function evaluate what is the best action to take based on previous experiences and rewards. Sometimes, it returns a random action to explore the environment.

Epsilon: Epsilon is the exploration rate. To be able to learn from previous experience, the agent has to start with random action. This rate can be small or equal to zero when testing because we may want to optimise exploitation. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #self.position[-2:, :] = -1
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0: 
            if self.x == self.grid_size-3:
                self.x = self.x-1               # if  edge, go UP 
            else:
                self.x = self.x + 1             # go DOWN
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1               # if  edge, go DOWN
            else:
                self.x = self.x-1               # go UP 
        elif action == 2:
            if self.y == self.grid_size - 3: 
                self.y = self.y - 1             # if  edge, go LEFT
            else:
                self.y = self.y + 1             # go RIGHT
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1             # if  edge, go RIGHT
            else:
                self.y = self.y - 1             # go LEFT
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T= 200
temperature = 0.3
epochs_train = 10 # set small when debugging
epochs_test = 5 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

Answer:
- 'Position' array gives the position (x,y) of the agent on the grid, 
- 'board' array gives all the possible positions on the board.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
import random

class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        
        
        pass

    def learned_act(self, s):
        return random.randint(0,3)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):

        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

            
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 8.5/9.0. Average score (-0.5)
Win/lose count 11.5/12.0. Average score (-0.5)
Win/lose count 9.5/25.0. Average score (-5.5)
Win/lose count 7.0/14.0. Average score (-5.875)
Win/lose count 10.0/12.0. Average score (-5.1)
Final score: -5.1


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




1) First part

According to Bellman's formula, we know that:
$$
Q^{\pi}(s,a)=E_{\pi}[\sum_{t=0}^{\infty}\gamma^tr_t(s_t,\pi(s_t),s_{t+1})|s_t=s,a_t=a]
$$
Hence:
\begin{array}{ll}
Q^{\pi}(s_0,a_0) = E_{\pi}[ r_0(s_0,a_0,s_1) + \sum_{t=1}^{\infty}\gamma^tr_t(s_t,\pi(s_t),s_{t+1})|s_0,a_0]\\
\\
 = E_{\pi}[ r_0(s_0,a_0,s_1) + \gamma E_{\pi}(\sum_{t=1}^{\infty}\gamma^{t-1}r_t(s_t,\pi(s_t),s_{t+1})|s_1,a_1)|s_0,a_0]\\
 \\
 = E_{\pi}[ r_0(s_0,a_0,s_1) + _\gamma Q^{\pi}(s_1,a_1)]
\end{array} 
Thus:
$$
\displaystyle Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
$$


2) Second part
$$
\displaystyle  V^{\pi}(s_0)=E[\sum_{t=0}^{\infty}\gamma^tr_t(s_t,\pi(s_t),s_{t+1}|s_0]
$$
and:
$$
\displaystyle V^*(s_0)=\max_{\pi,a_0} E[\sum_{t=0}^{\infty}\gamma^tr_t(s_t,\pi(s_t),s_{t+1}|s_0]
$$
Thus:
\begin{array}{ll}
\displaystyle V^*(s_0)=\max_{\pi,a_0} E[r_0(s_0,a_0,s_1) + \sum_{t=1}^{\infty}\gamma^tr_t(s_t,\pi(s_t),s_{t+1}|s_0]\\
\\
\displaystyle = \max_{a_0} E_{s_1}[r_0(s_0,a_0,s_1) + \gamma \max_{\pi} E_{s_2,s_3,...} [\sum_{t=1}^{\infty}\gamma^{t-1}r_t(s_t,\pi(s_t),s_{t+1}]|s_0]\\
 \\
\displaystyle = \max_{a_0}E_{s_1}[r_0(s_0,a_0,s_1)+\gamma V^*(s_1)]|\\
 \\
\displaystyle = V^*(s) = \max E_{s'}[r(s,a,s')+\gamma V^*(s')|s]
\end{array} 
We know that:$\displaystyle V^*(s)=\max_{a'} Q^*(s,a')$
Thus:
$$
\displaystyle V^*(s) = \max_a E_{s'}[r(s,a,s')+\gamma \max_{a'} Q^*(s',a')|s]
$$
And we have then:
\begin{array}{ll}
\displaystyle Q^*(s,a) = E_{s'}[r(s,a,s')+\gamma \max_{a'} Q^*(s',a')|s]
\\
\displaystyle = E_{s' \sim \pi^*(.|s,a)}[r(s,a,s')+\gamma \max_{a'} Q^*(s',a')]
\end{array} 

3) Third part
Loss function formula:
$$
L(\theta)=[Q^*(s,a,\theta)-Q(s,a,\theta)]^2
$$
Then, replacing $Q^*(s,a,\theta)$, we have:
\begin{array}{ll}
\displaystyle L(\theta) = E_{s,a}[ (E_{s'}(r(s,a,s')+\gamma \max_{a'} Q(s',a',\theta))-Q(s,a,\theta))^2]\\
\\
 = E_{s,a}[(\textbf{Target(critic) - actor})^2]
\end{array} 


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        
        if len(self.memory) > self.max_memory:
            del self.memory[0]
        
        pass

    def random_access(self):
        mini_batch = random.choice(self.memory)
        return mini_batch

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size, epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__()

        # Discount for Q learning
        self.discount = 0.8
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

               

    def learned_act(self, s):
        
        act = self.model.predict(np.array([s],ndmin=2)) 
        return np.argmax(act[0])

    def reinforce(self, s_, n_s, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4)) # 4 actions to take
        
        
        #for state, action, reward, next_state, done in minibatch:
        for i in range(self.batch_size):
            
            # we create a mini batch constituted of random past experiences:
            mini_batch = self.memory.random_access()
            
            # each row of mini_batch has: state, next_state, action, reward, game_over_
            s_ , n_s_, a_, r_, game_over_ = mini_batch

            input_states[i] = s_
            
            if game_over_:
                target_q[i, a_] = r_ # no more possible state
                
            else:
                x = np.reshape(n_s_, (1, 5, 5, self.n_state)) 
                target_q[i, a_] = r_ + self.discount * np.amax(self.model.predict(x)) 
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q) # loss function. 


        
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        model.add(Reshape((5*5*self.n_state,), input_shape=(5,5,self.n_state)))
        model.add(Dense(64, input_dim=5*5*self.n_state))
        model.add(Dense(4))
        
        model.compile(sgd(lr=0.1, decay=0.0001, momentum=0.0), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=0.1, epsilon = 0.1, memory_size=2000, batch_size = 32)

epochs_train = 10
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train0.mp4'))

Epoch 000/010 | Loss 0.0171 | Win/lose count 4.5/1.0 (3.5)
Epoch 001/010 | Loss 0.0052 | Win/lose count 5.5/4.0 (1.5)
Epoch 002/010 | Loss 0.0050 | Win/lose count 4.0/3.0 (1.0)
Epoch 003/010 | Loss 0.0025 | Win/lose count 5.5/4.0 (1.5)
Epoch 004/010 | Loss 0.0093 | Win/lose count 6.5/2.0 (4.5)
Epoch 005/010 | Loss 0.0107 | Win/lose count 1.5/1.0 (0.5)
Epoch 006/010 | Loss 0.0045 | Win/lose count 10.5/3.0 (7.5)
Epoch 007/010 | Loss 0.0124 | Win/lose count 13.5/6.0 (7.5)
Epoch 008/010 | Loss 0.0102 | Win/lose count 12.0/4.0 (8.0)
Epoch 009/010 | Loss 0.0020 | Win/lose count 12.5/5.0 (7.5)


***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
from keras.layers import Flatten

class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(64, (2, 2), input_shape=(5, 5, self.n_state)))
        model.add(Conv2D(16, (2, 2)))
        
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, 10, prefix='cnn_train')
HTML(display_videos('cnn_train0.mp4'))

Epoch 000/010 | Loss 0.0168 | Win/lose count 8.0/4.0 (4.0)
Epoch 001/010 | Loss 0.0089 | Win/lose count 4.5/7.0 (-2.5)
Epoch 002/010 | Loss 0.0059 | Win/lose count 7.0/4.0 (3.0)
Epoch 003/010 | Loss 0.0243 | Win/lose count 7.0/7.0 (0.0)
Epoch 004/010 | Loss 0.0079 | Win/lose count 4.5/6.0 (-1.5)
Epoch 005/010 | Loss 0.0094 | Win/lose count 11.5/2.0 (9.5)
Epoch 006/010 | Loss 0.0023 | Win/lose count 0/1.0 (-1.0)
Epoch 007/010 | Loss 0.0086 | Win/lose count 7.5/2.0 (5.5)
Epoch 008/010 | Loss 0.0062 | Win/lose count 3.0/2.0 (1.0)
Epoch 009/010 | Loss 0.0090 | Win/lose count 8.5/1.0 (7.5)


***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [17]:
env = Environment(grid_size=size, max_time=T,temperature=0.5)
epochs_test = 5

agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

print('Test of the CNN')
test(agent_cnn, env, epochs_test, prefix='cnn_test')

print('Test of the FC')
test(agent_fc, env, epochs_test, prefix='fc_test')

Test of the CNN
Win/lose count 6.0/2.0. Average score (4.0)
Win/lose count 13.0/5.0. Average score (6.0)
Win/lose count 1.0/3.0. Average score (3.3333333333333335)
Win/lose count 8.0/1.0. Average score (4.25)
Win/lose count 6.0/7.0. Average score (3.2)
Final score: 3.2
Test of the FC
Win/lose count 8.5/9.0. Average score (-0.5)
Win/lose count 8.0/5.0. Average score (1.25)
Win/lose count 7.0/0. Average score (3.1666666666666665)
Win/lose count 7.5/1.0. Average score (4.0)
Win/lose count 8.5/4.0. Average score (4.1)
Final score: 4.1


In [18]:
HTML(display_videos('cnn_test0.mp4'))

In [19]:
HTML(display_videos('fc_test0.mp4'))

In [20]:
env = Environment(grid_size=size, max_time=T,temperature=0.9)
epochs_test = 5

agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

print('Test of the CNN')
test(agent_cnn, env, epochs_test, prefix='cnn_test')

print('Test of the FC')
test(agent_fc, env, epochs_test, prefix='fc_test')

Test of the CNN
Win/lose count 9.0/3.0. Average score (6.0)
Win/lose count 7.5/2.0. Average score (5.75)
Win/lose count 10.5/1.0. Average score (7.0)
Win/lose count 10.5/1.0. Average score (7.625)
Win/lose count 10.5/0. Average score (8.2)
Final score: 8.2
Test of the FC
Win/lose count 10.5/4.0. Average score (6.5)
Win/lose count 24.0/4.0. Average score (13.25)
Win/lose count 24.0/3.0. Average score (15.833333333333334)
Win/lose count 35.0/4.0. Average score (19.625)
Win/lose count 31.0/2.0. Average score (21.5)
Final score: 21.5


In [22]:
env = Environment(grid_size=size, max_time=T,temperature=0.1)
epochs_test = 5

agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

print('Test of the CNN')
test(agent_cnn, env, epochs_test, prefix='cnn_test')

print('Test of the FC')
test(agent_fc, env, epochs_test, prefix='fc_test')

Test of the CNN
Win/lose count 3.5/0. Average score (3.5)
Win/lose count 0.5/1.0. Average score (1.5)
Win/lose count 3.0/2.0. Average score (1.3333333333333333)
Win/lose count 0/0. Average score (1.0)
Win/lose count 3.5/3.0. Average score (0.9)
Final score: 0.9
Test of the FC
Win/lose count 4.5/6.0. Average score (-1.5)
Win/lose count 0/0. Average score (-0.75)
Win/lose count 1.0/0. Average score (-0.16666666666666666)
Win/lose count 2.0/1.0. Average score (0.125)
Win/lose count 5.5/2.0. Average score (0.8)
Final score: 0.8


Answer:
- We observe that at temperature = 0.5, both models tends to perform the same (CNN is slightly better)
- When temperature increases, FC seems to perform better (probably because the bonus/malus is more significant)
- When the temprature is low, both agents have difficulty to reach the cheese (probably because the bonus/malus is less significant)

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [24]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            #state, reward, game_over = env.act(action)
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)
            
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e+1, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        # define borders
        self.position[0:2,:]= -1 # first two lines
        self.position[:,0:2] = -1 # first two columns
        self.position[-2:, :] = -1 # last two lines
        self.position[:,-2:] = -1 # last two columns

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward = reward + self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0] = 0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [26]:
# Training

env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, 10, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore0.mp4'))

Epoch 001/010 | Loss 0.0171 | Win/lose count 2.5/25.600000000000065 (-23.100000000000065)
Epoch 002/010 | Loss 0.0047 | Win/lose count 3.5/18.19999999999999 (-14.699999999999989)
Epoch 003/010 | Loss 0.0147 | Win/lose count 13.000000000000002/22.100000000000023 (-9.100000000000021)
Epoch 004/010 | Loss 0.0070 | Win/lose count 9.4/22.400000000000066 (-13.000000000000066)
Epoch 005/010 | Loss 0.0214 | Win/lose count 5.000000000000001/20.600000000000012 (-15.600000000000012)
Epoch 006/010 | Loss 0.0117 | Win/lose count 5.4/20.300000000000004 (-14.900000000000004)
Epoch 007/010 | Loss 0.0124 | Win/lose count 12.900000000000006/18.8 (-5.899999999999995)
Epoch 008/010 | Loss 0.0175 | Win/lose count 4.3999999999999995/22.600000000000062 (-18.200000000000063)
Epoch 009/010 | Loss 0.0053 | Win/lose count 10.400000000000004/18.499999999999996 (-8.099999999999993)
Epoch 010/010 | Loss 0.0034 | Win/lose count 9.500000000000004/20.600000000000012 (-11.100000000000009)


In [27]:
# Evaluation
test(agent,env,5,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore0.mp4'))

Win/lose count 13.200000000000006/23.800000000000054. Average score (-10.600000000000048)
Win/lose count 11.300000000000004/19.300000000000004. Average score (-9.300000000000024)
Win/lose count 12.400000000000006/22.000000000000057. Average score (-9.400000000000032)
Win/lose count 11.200000000000005/23.30000000000006. Average score (-10.075000000000038)
Win/lose count 7.200000000000002/19.300000000000008. Average score (-10.480000000000032)
Final score: -10.48


***